In [1]:
# sample table, load as df 

# https://en.wikipedia.org/wiki/The_World%27s_Billionaires
# https://medium.com/intel-tech/tabular-data-rag-llms-improve-results-through-data-table-prompting-bcb42678914b

import pandas as pd
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://en.wikipedia.org/wiki/The_World%27s_Billionaires")
data = loader.load()

data


USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://en.wikipedia.org/wiki/The_World%27s_Billionaires', 'title': "The World's Billionaires - Wikipedia", 'language': 'en'}, page_content='\n\n\n\nThe World\'s Billionaires - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sideb

In [7]:
#file_path = "./The World's Billionaires - Wikipedia.pdf"

file_path = "./World Tourism rankings - Wikipedia.pdf"

In [12]:
import camelot
from typing import List
# use camelot to parse tables   
def get_tables(path: str, pages: List[int]):    
    for page in pages:
        table_list = camelot.read_pdf(path, pages=str(page))
        if table_list.n>0:
            for tab in range(table_list.n):
                
                # Conversion of the the tables into the dataframes.
                table_df = table_list[tab].df 
                
                table_df = (
                    table_df.rename(columns=table_df.iloc[0])
                    .drop(table_df.index[0])
                    .reset_index(drop=True)
                )        
                     
                table_df = table_df.apply(lambda x: x.str.replace('\n',''))
                
                # Change column names to be valid as XML tags
                table_df.columns = [col.replace('\n', ' ').replace(' ', '') for col in table_df.columns]
                table_df.columns = [col.replace('(', '').replace(')', '') for col in table_df.columns]
    
    return table_df
# extract data table from page number
df = get_tables(file_path, pages=[1])

In [13]:
df

,Rank,Destination,Internationaltouristarrivals2023[4],Internationaltouristarrivals2022[3],Change2022to2023%,Change2021to2022%
0,1,France,100 million,79.4 million,25.9,64
1,2,Spain,85.2 million,71.7 million,18.8,130
2,3,United States,66.5 million,50.9 million,30.6,130
3,4,Italy,57.2 million,49.8 million,14.8,85
4,5,Turkey,55.2 million,50.5 million,9.3,69
5,6,Mexico,42.2 million,38.3 million,10.1,20
6,7,United Kingdom,37.2 million,30.7 million,21.1,389
7,8,Germany,34.8 million,28.5 million,22.1,144
8,9,Greece,32.7 million,27.8 million,17.6,89
9,10,Austria,30.9 million,26.2 million,17.9,106


In [15]:
str_data = df.to_string(index=False)
str_data

'Rank    Destination Internationaltouristarrivals2023[4] Internationaltouristarrivals2022[3] Change2022to2023% Change2021to2022%\n   1         France                         100 million                        79.4 million              25.9                64\n   2          Spain                        85.2 million                        71.7 million              18.8               130\n   3  United States                        66.5 million                        50.9 million              30.6               130\n   4          Italy                        57.2 million                        49.8 million              14.8                85\n   5         Turkey                        55.2 million                        50.5 million               9.3                69\n   6         Mexico                        42.2 million                        38.3 million              10.1                20\n   7 United Kingdom                        37.2 million                        30.7 million     

In [16]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ChatMessageHistory
import gradio as gr

llm = ChatOllama(model="llama3")

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions using this data: \n\n{data}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | llm

chat_history = ChatMessageHistory()

def respond(message, history):
    chat_history.add_user_message(message)
    response = chain.invoke({"data": str_data, "messages": chat_history.messages})
    chat_history.add_ai_message(response.content)
    return "Bot: " + response.content

chat_interface = gr.ChatInterface(fn=respond)
chat_interface.launch()

/Users/adudi/Documents/projects/ollama_agent/venv/lib/python3.10/site-packages/gradio/components/chatbot.py:288: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
